In [1]:
from datasets import load_dataset

In [29]:
corpus = load_dataset("HydraLM/SkunkData-Corpus-Clusters-001")


Extracting data files:   0%|          | 0/32 [00:00<?, ?it/s]

Generating config0 split:   0%|          | 0/99425 [00:00<?, ? examples/s]

Generating config1 split:   0%|          | 0/125333 [00:00<?, ? examples/s]

Generating config2 split:   0%|          | 0/142226 [00:00<?, ? examples/s]

Generating config3 split:   0%|          | 0/95365 [00:00<?, ? examples/s]

...

Generating config27 split:   0%|          | 0/61803 [00:00<?, ? examples/s]

Generating config28 split:   0%|          | 0/100282 [00:00<?, ? examples/s]

Generating config29 split:   0%|          | 0/87382 [00:00<?, ? examples/s]

Generating config30 split:   0%|          | 0/31757 [00:00<?, ? examples/s]

Generating config31 split:   0%|          | 0/58432 [00:00<?, ? examples/s]

In [31]:
corpus

DatasetDict({
    config0: Dataset({
        features: ['message', 'message_type', 'message_id', 'conversation_id', 'dataset_id', 'unique_conversation_id', 'cluster', '__index_level_0__'],
        num_rows: 99425
    })
    config1: Dataset({
        features: ['message', 'message_type', 'message_id', 'conversation_id', 'dataset_id', 'unique_conversation_id', 'cluster', '__index_level_0__'],
        num_rows: 125333
    })
    config2: Dataset({
        features: ['message', 'message_type', 'message_id', 'conversation_id', 'dataset_id', 'unique_conversation_id', 'cluster', '__index_level_0__'],
        num_rows: 142226
    })
    config3: Dataset({
        features: ['message', 'message_type', 'message_id', 'conversation_id', 'dataset_id', 'unique_conversation_id', 'cluster', '__index_level_0__'],
        num_rows: 95365
    })
    config4: Dataset({
        features: ['message', 'message_type', 'message_id', 'conversation_id', 'dataset_id', 'unique_conversation_id', 'cluster', '__inde

In [32]:
corpus = corpus.remove_columns("__index_level_0__")

In [33]:
corpus

DatasetDict({
    config0: Dataset({
        features: ['message', 'message_type', 'message_id', 'conversation_id', 'dataset_id', 'unique_conversation_id', 'cluster'],
        num_rows: 99425
    })
    config1: Dataset({
        features: ['message', 'message_type', 'message_id', 'conversation_id', 'dataset_id', 'unique_conversation_id', 'cluster'],
        num_rows: 125333
    })
    config2: Dataset({
        features: ['message', 'message_type', 'message_id', 'conversation_id', 'dataset_id', 'unique_conversation_id', 'cluster'],
        num_rows: 142226
    })
    config3: Dataset({
        features: ['message', 'message_type', 'message_id', 'conversation_id', 'dataset_id', 'unique_conversation_id', 'cluster'],
        num_rows: 95365
    })
    config4: Dataset({
        features: ['message', 'message_type', 'message_id', 'conversation_id', 'dataset_id', 'unique_conversation_id', 'cluster'],
        num_rows: 160680
    })
    config5: Dataset({
        features: ['message', 'mess

In [34]:
def user_assistant_template(conversation, max_words=1400):
    # sort by message_id
    conversation = sorted(conversation, key=lambda x: x["message_id"])
    output = f"{conversation.pop()['message']}"
    input = "### User:"

    for message in conversation:
        if message["message_type"] == "instruction" or message["message_type"] == "input":
            input += f"\n{message['message'].strip()}" if message['message'].strip() != "" else ""

        elif message["message_type"] == "output":
            input += f"\n\n### Assistant:\n{message['message']}\n\n### User:"

    input += "\n\n### Assistant:\n"

    if len(input.split()) > max_words:
        input = " ".join(input.split(" ")[-max_words:])

    instruction_start = input.find("### User:")
    input = input[instruction_start:]
    return [{"input": input, "output": output}]

In [37]:
from datasets import Dataset
import pandas as pd

def sort_Dataset_by_multiple_keys(dataset, keys):
    df = dataset.to_pandas()
    df = df.sort_values(by=keys)
    return Dataset.from_pandas(df)


def format(path_or_dataset, template_func, filter_key="unique_conversation_id",
           sort_keys=["dataset_id", "conversation_id", "message_id"], split=None):
    dataset = load_dataset(path_or_dataset) if isinstance(path_or_dataset, str) else path_or_dataset
    data = dataset[split] if split else dataset
    data = sort_Dataset_by_multiple_keys(data, sort_keys)

    formatted = []
    current_filter_key = data[0][filter_key]
    conversation = []

    for message in data:
        if message[filter_key] != current_filter_key:
            formatted += template_func(conversation)
            current_filter_key = message[filter_key]
            conversation = []
        conversation += [message]

    return Dataset.from_pandas(pd.DataFrame(data=formatted))


In [38]:
from datasets import DatasetDict, Dataset
new_corpus = DatasetDict()
for cluster_dataset in corpus:
    new_corpus[cluster_dataset] = format(corpus[cluster_dataset], user_assistant_template, filter_key="unique_conversation_id")

In [39]:
new_corpus

DatasetDict({
    config0: Dataset({
        features: ['input', 'output'],
        num_rows: 45617
    })
    config1: Dataset({
        features: ['input', 'output'],
        num_rows: 57017
    })
    config2: Dataset({
        features: ['input', 'output'],
        num_rows: 59271
    })
    config3: Dataset({
        features: ['input', 'output'],
        num_rows: 41544
    })
    config4: Dataset({
        features: ['input', 'output'],
        num_rows: 72587
    })
    config5: Dataset({
        features: ['input', 'output'],
        num_rows: 34080
    })
    config6: Dataset({
        features: ['input', 'output'],
        num_rows: 30203
    })
    config7: Dataset({
        features: ['input', 'output'],
        num_rows: 36365
    })
    config8: Dataset({
        features: ['input', 'output'],
        num_rows: 82654
    })
    config9: Dataset({
        features: ['input', 'output'],
        num_rows: 34266
    })
    config10: Dataset({
        features: ['input', 'out

In [42]:
for value in new_corpus["config0"][0].values():
    print(value)

### User:
Dog-friendly hiking trails

### Assistant:

Sure, I can certainly provide some information on dog-friendly hiking trails, though the specific trails would depend on your location. Here are few notable ones across the United States:

1. Acadia National Park, Maine: Most trails in this national park on the rugged coast of Maine are dog-friendly. The park boasts 100 miles of hiking trails and 45 miles of carriage roads where dogs are allowed. 

2. Runyon Canyon Park, Los Angeles, California: This 160-acre park right in the heart of LA is very popular with dog owners. It has off-leash areas so that your pup can run free, and it offers stunning views of the city. 

3. The Appalachian Trail, Eastern US: This iconic trail, which stretches from Georgia to Maine, allows dogs on most of its paths, but they must be leashed.

4. Shenandoah National Park, Virginia: Home to 500 miles of trails, Shenandoah is one of the most dog-friendly national parks. Dogs are allowed on nearly all of the

In [43]:
new_corpus.push_to_hub("HydraLM/corpus_1_clustered_formatted")

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/46 [00:00<?, ?ba/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/58 [00:00<?, ?ba/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

...

Creating parquet from Arrow format:   0%|          | 0/33 [00:00<?, ?ba/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/16 [00:00<?, ?ba/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/27 [00:00<?, ?ba/s]

In [44]:
# Exploring examples of formatted data
df = pd.DataFrame(new_corpus["config0"])
for i in range(10):
    print("********************************INPUT********************************")
    print(df.iloc[i]["input"])
    print("********************************OUTPUT********************************")
    print(df.iloc[i]["output"])
    print("=====================================")

********************************INPUT********************************
### User:
Dog-friendly hiking trails

### Assistant:

********************************OUTPUT********************************
Sure, I can certainly provide some information on dog-friendly hiking trails, though the specific trails would depend on your location. Here are few notable ones across the United States:

1. Acadia National Park, Maine: Most trails in this national park on the rugged coast of Maine are dog-friendly. The park boasts 100 miles of hiking trails and 45 miles of carriage roads where dogs are allowed. 

2. Runyon Canyon Park, Los Angeles, California: This 160-acre park right in the heart of LA is very popular with dog owners. It has off-leash areas so that your pup can run free, and it offers stunning views of the city. 

3. The Appalachian Trail, Eastern US: This iconic trail, which stretches from Georgia to Maine, allows dogs on most of its paths, but they must be leashed.

4. Shenandoah National 